# CMS Processor
This notebook can combine all CMS for all semesters and programme leaders can have a clear picture of your programme.

In [ ]:
%pip install pandas xlrd xlwt openpyxl xlsxwriter

In [ ]:
# read all excel files in xls in folder data/, then save all file name and sheet name as dict of list
import pandas as pd 
import os

# read all excel files in folder data/
path = 'data/'
files = os.listdir(path)
files_xls = [f for f in files if f[-3:] == 'xls']

# save all file name and sheet name as dict of list
file_sheets = {}
for f in files_xls:
    file_sheets[f] = pd.ExcelFile(path + f).sheet_names
    
# remove 'selection criteria', 'CMS Notes for Users', 'Overall Grade Desc'and 'Enrichment Module List' from dict value.
for k, v in file_sheets.items():
    print(k)
    v.remove('selection criteria')
    v.remove('CMS Notes for Users')
    v.remove('Overall Grade Desc')
    v.remove('Enrichment Module List')

file_sheets   

In [ ]:
def extractMarkTable(file_path, sheet_name):
    import xlrd      
    wb = xlrd.open_workbook(file_path)  
    sheet = wb[sheet_name]   

    # Extract cell values  
    programme = sheet.cell_value(3, 1)  
    class_ = sheet.cell_value(6, 3)  
    academic_year = sheet.cell_value(2, 1)  
    semester = sheet.cell_value(2, 3)  
    join_year = sheet.cell_value(5, 1)  
    join_semester = sheet.cell_value(5, 3)  
    campus = sheet.cell_value(4, 1) 
    target_award = sheet.cell_value(6, 1) 
    
    # Find the starting row and last column  
    start_row, last_column = None, None  
    for row in range(sheet.nrows):  
        for col in range(sheet.ncols):  
            cell_value = sheet.cell_value(row, col)  
            if cell_value == "Seq No.":  
                start_row = row  
            if cell_value == "CMS Remark":  
                last_column = col  
            if start_row and last_column is not None:  
                break  
        if start_row and last_column is not None:  
            break  
    
    # Find the last row with a value in the last column  
    last_row = None  
    for row in range(start_row, sheet.nrows):  
        cell_value = sheet.cell_value(row, last_column)  
        if cell_value == "":  
            break  
        last_row = row  
    
    # Save the region to a 2D array  
    data = []  
    for row in range(start_row, last_row + 1):  
        data_row = [sheet.cell_value(row, col) for col in range(0, last_column + 1)]  
        data.append(data_row)  
    
    # Convert the 2D array to a pandas DataFrame  
    column_names = data.pop(0)  # Remove the first row as column names  
    df = pd.DataFrame(data, columns=column_names)  

    # Add extracted cell values to the DataFrame as columns  
    df['Programme'] = programme  
    df['Class'] = class_  
    df['Academic Year'] = academic_year  
    df['Semester'] = semester.split(' ')[1]  
    df['Join Year'] = join_year  
    df['Join Semester'] = join_semester.split(' ')[1]    
    df['Campus'] = campus  
    df['Target Award'] = target_award
    df.drop(columns='Seq No.', inplace=True)  
    df.set_index('Student No.', inplace=True)
    df = df.reset_index()

   
    if len(df.columns) != len(set(df.columns)):        
        # remove duplicate column name
        df = df.loc[:,~df.columns.duplicated("first")]   

    return df, join_year + campus + target_award+ class_ + academic_year + semester

# df, name = extractMarkTable('data/2223 Sem 1 115 cms for boe.xls', '20IVE(LW)1IT114115-HD3R')
# name

marksheets = []
for k, v in file_sheets.items():
    for sheet_name in v:
        df, name = extractMarkTable(path + k, sheet_name)
        marksheets.append(df)
        df.to_excel('output/' + name + '.xlsx', index=False)



In [ ]:
marksheets

In [ ]:
# Put all student id in a list from all marksheet
student_ids = []
for df in marksheets:
    student_ids.extend(df['Student No.'].tolist())
student_ids.sort()
# remove duplicate student id
student_ids = list(set(student_ids))

In [ ]:
#get student records from all marksheets for each ID.

unique_keys = set()
student_records = {}
for id in student_ids:
    student_records[id] = []
    for df in marksheets:
        #get row with student id
        student_record = df.loc[df['Student No.'] == id]
        if not student_record.empty:
            #convert row to dict
            student_record = student_record.to_dict('records')[0]
            # #remove student id from dict
            student_record.pop('Student No.')
            # if dict key contains "\n", split it and take the first part.
            for k in list(student_record.keys()):
                if "\n" in k:
                    student_record[k.split("\n")[0]] = student_record[k]
                    student_record.pop(k)
            #add dict to list            
            student_records[id].append(student_record)
            unique_keys.update(student_record.keys())

remove_list = ["Programme","% of Credit Failed","Academic Year","CGPA/CMA","CMS Remark","Campus","Class","Student Pool","Target Award",
               "Total Credits Exempted","Total Credits Failed","Total Credits Taken",'Remaining MRP    (No of Semester)','Semester',
               'Join Semester','Join Year','Entry Level ','Overall Grade','Student Name','SGPA/SMA']
#remove key  from unique_keys in remove_list
unique_keys = [x for x in unique_keys if x not in remove_list]
unique_keys.sort()
unique_keys


In [ ]:
# Sort the value list in student_records by Academic Year then Semester in accending order.
student_summeries = {}
for k, v in student_records.items():
    student_records[k] = sorted(v, key=lambda x: (x['Academic Year'], x['Semester']), reverse=False)
    # merge all dict in list to one dict. if there is duplicte key, take the first dict in the list.
    student_summeries[k] = {k: v for d in student_records[k] for k, v in d.items()}       


In [ ]:
#load Modules.xlsx into dataframe
modules = pd.read_excel('Modules.xlsx')
# In modules dataframe, if "Module Code" duplicated, take the last row.
modules = modules.drop_duplicates(subset=['Module Code'], keep='last')
for index, row in modules.iterrows():
    if "," in row['Semester']:
        row['Semester'] = row['Semester'].split(",")[0].split(" ")[0] + " "+row['Semester'].split(",")[-1]
    if "," in row['E&L Semester']:
        row['E&L Semester'] = row['E&L Semester'].split(",")[0].split(" ")[0] + " "+row['E&L Semester'].split(",")[-1]
# trim module code and E&L module code 
modules['Module Code'] = modules['Module Code'].str.strip()
modules['E&L Module Code'] = modules['E&L Module Code'].str.strip()

# Get distinct Semester from modules dataframe
semesters = modules['Semester'].unique().tolist()
semesters.sort()

# Get semeester to module code mapping
semester_module_mapping = {}
for semester in semesters:
    semester_module_mapping[semester] = modules.loc[modules['Semester'] == semester]['Module Code'].tolist()
    semester_module_mapping[semester].sort()


semester_module_mapping



In [ ]:
# Create Excel file for student_ids in the first column and the rest of column 
# 'Student Name', 'Class', 'CGPA/CMA', then followed 'Module Code' in modules dataframe.
# The value of each cell is from student_summeries dict key is student ID and value is dict of student record key and value where key is in column.

import xlsxwriter
workbook = xlsxwriter.Workbook('output/Student Raw Summary.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write(0, 0, 'Student ID')
worksheet.write(0, 1, 'Student Name')
worksheet.write(0, 2, 'Class')
worksheet.write(0, 3, 'Join Year')
worksheet.write(0, 4, 'CGPA/CMA')
col = 5
for index, row in modules.iterrows():
    worksheet.write(0, col, row['Module Code'])
    col += 1

row = 1
for id in student_ids:
    worksheet.write(row, 0, id)
    worksheet.write(row, 1, student_summeries[id]['Student Name'])
    worksheet.write(row, 2, student_summeries[id]['Class'])
    worksheet.write(row, 3, student_summeries[id]['Join Year'])
    worksheet.write(row, 4, student_summeries[id]['CGPA/CMA'])
    col = 5
    for index, rowDf in modules.iterrows():
        if rowDf['Module Code'] in student_summeries[id]:
            value = student_summeries[id][rowDf['Module Code']]            
            worksheet.write(row, col, value)
        col += 1
    row += 1

workbook.close()

In [ ]:
# load  Student Raw Summary.xlsx into dataframe
import math
import numpy as np


student_summary = pd.read_excel('output/Student Raw Summary.xlsx')
# for all cell if start with single "\n" followed a string, remove the "\n".
for index, row in student_summary.iterrows():
    for col in student_summary.columns:
        if isinstance(row[col], str) and row[col].startswith("\n"):
            student_summary.at[index, col] = row[col][1:]

# for Enrichment A and Enrichment B columns, if the cell contains "\n", split it and take the last part.
for index, row in student_summary.iterrows():
    if isinstance(row['Enrichment A'], str) and "\n" in row['Enrichment A']:
        student_summary.at[index, 'Enrichment A'] = row['Enrichment A'].split("\n")[-1]
    if isinstance(row['Enrichment B'], str) and "\n" in row['Enrichment B']:
        student_summary.at[index, 'Enrichment B'] = row['Enrichment B'].split("\n")[-1]

# sort student_summary by Class then Student Name in accending order and reset the index.
student_summary = student_summary.sort_values(by=['Join Year','Class', 'Student Name'], ascending=[True, True, True])
student_summary = student_summary.reset_index(drop=True)

def fail_enrichment_b(value):       
    if isinstance(value, str):       
        return value != "P"
    return False
      
def fail_enrichment_a(value):          
    if isinstance(value, str):          
        return "<" in value
    return False
    
# Create new column "No of F" for the sum of row cell contains "\n" in the cell value.
student_summary['Total F'] = student_summary.apply(lambda x: len([i for i in x if isinstance(i, str) and "\n" in i]), axis=1)
# If Enrichment B has value is not equal to 'P', 'Total F' plus 1.
student_summary['Total F'] = student_summary.apply(lambda x: x['Total F'] + 1 if fail_enrichment_b(x['Enrichment B']) else x['Total F'], axis=1)
# If Enrichment A contains "<", 'Total F' plus 1.
student_summary['Total F'] = student_summary.apply(lambda x: x['Total F'] + 1 if fail_enrichment_a(x['Enrichment A']) else x['Total F'], axis=1)

# iterate semester_module_mapping, count 'Total F' for each semester and save the value in new column.
for semester, modules in semester_module_mapping.items():
    semester = semester + " F"
    student_summary[semester] = 0
    student_summary[semester] = student_summary.apply(lambda x: len([i for i in modules if isinstance(x[i], str) and "\n" in x[i]]), axis=1)
    if 'Enrichment B' in modules:
        student_summary[semester] = student_summary.apply(lambda x: x[semester] + 1 if fail_enrichment_b(x['Enrichment B']) else x[semester], axis=1)
    if 'Enrichment A' in modules:
        student_summary[semester] = student_summary.apply(lambda x: x[semester] + 1 if fail_enrichment_a(x['Enrichment A']) else x[semester], axis=1)

for semester, modules in semester_module_mapping.items():
    semester = semester + " ABV"
    student_summary[semester] = 0
    student_summary[semester] = student_summary.apply(lambda x: len([i for i in modules if isinstance(x[i], str) and "ABV" in x[i] in x[i]]), axis=1)

number_of_F_columns = len(semester_module_mapping) * 2 + 1
# move the last number_of_F_columns columns to the firth of dataframe.
cols = list(student_summary.columns)
cols = cols[:5] + cols[-number_of_F_columns:] + cols[5:-number_of_F_columns]
student_summary = student_summary[cols]

In [ ]:
# Use 2D loop to export student_summary to excel file with multiple sheets for mulitple dataframe.
# Iterate through all rows in dataframe, if the cell contains multple line highligh the cell with red.

import xlsxwriter
workbook = xlsxwriter.Workbook('output/Student Summary.xlsx')
    # Add a worksheet for each student
worksheet = workbook.add_worksheet("Summary")

# Add a format. Light red fill with dark red text.
format1 = workbook.add_format({'bg_color': '#FFC7CE',
                                 'font_color': '#9C0006'})
# Add a format. Light red fill with dark green text.
format2 = workbook.add_format({'bg_color': '#C6EFCE',
                                    'font_color': '#006100'})

# student_summary column name is the first row
worksheet.write_row(0, 0, student_summary.columns.tolist())

# save Enrichment A column index to enrichment_a_index
enrichment_a_index = student_summary.columns.tolist().index('Enrichment A')
# save Enrichment B column index to enrichment_b_index
enrichment_b_index = student_summary.columns.tolist().index('Enrichment B')

student_summary.fillna('', inplace=True)
for index, row in student_summary.iterrows():
    # Iterate over the data and write it out row by row.
    for col_num, value in enumerate(row):
        # for cell value contain "ABV", highlight the cell with green.
        if isinstance(value, str) and "ABV" in value:           
            worksheet.write(index + 1, col_num, value, format2)
            continue     
        # for Enrichment B column, if the cell value is not equal to 'P', highlight the cell with red.
        if col_num == enrichment_b_index and not (value == 'P' or value == ''):
            worksheet.write(index + 1, col_num, value, format1)
            continue
        # for Enrichment A column, if the cell value is not equal to '' and contains <, highlight the cell with red.
        if col_num == enrichment_a_index and (value != '' and "<" in value):
            worksheet.write(index + 1, col_num, value, format1)
            continue
        # for other column, if the cell value is string and contains \n, highlight the cell with red.
        if isinstance(value, str) and "\n" in value:
            worksheet.write(index + 1, col_num, value, format1)
            continue
   
        worksheet.write(index + 1, col_num, value)            


# add dynamic filter to the first row and freeze the first row
worksheet.autofilter(0, 0, 0, len(student_summary.columns.tolist()) - 1)
worksheet.freeze_panes(1, 0)

# auto column width and auto row height
for col_num, value in enumerate(student_summary.columns.tolist()):
    # set width based on the length of the column header
    worksheet.set_column(col_num, col_num, len(value) + 5)


for index, row in student_summary.iterrows():
    for col_num, value in enumerate(row):
        # set height based on the number of lines in the cell
        worksheet.set_row(index + 1, 15 * (len(str(value).split("\n")) + 1))
workbook.close()